### Libraries

In [ ]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.llms import Ollama
from langchain.chains import LLMChain
from transformers import pipeline
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import Chroma
from tqdm import tqdm
from langchain.schema import Document
from langchain.prompts import PromptTemplate
import chromadb
import numpy as np
import os
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

### Pre-processing

In [2]:
df = pd.read_csv(r"C:\Users\Chandler\Downloads\Misty\scheme_rag\updated_data.csv")
df.drop('Unnamed: 9', axis=1, inplace=True)
df.head(5)

,scheme_name,slug,details,benefits,eligibility,application,documents,level,schemeCategory,tags
0,"""Immediate Relief Assistance"" under ""Welfare a...",ira-wrflsncs,"The scheme ""Immediate Relief Assistance"" is a ...","₹ 1,00,000, in two installments of ₹ 50,000 ea...",The applicant should be the family (legal heir...,Step 1: The interested applicant should visit ...,Photograph of the Family (Legal Heir) of the M...,State,"Agriculture,Rural & Environment, Social welfar...","Missing, Fisherman, Relief, Financial Assistan..."
1,AICTE SHORT TERM TRAINING PROGRAMME-SFURTI SCHEME,astpss,"Short Term Training Programme-SFURTI Program, ...","Financial Assistance : Limit of funding ₹ 4,00...",The institution should be AICTE approved.,Registration of New Institute: Step 01: Visit ...,Feedback Form Copy of Proceedings Completion R...,Central,Education & Learning,"Trainings, Financial Assistance, AICTE"
2,Burial and Ex-gratia Payment Scheme in Case of...,baepsicodouldwact,"Launched in 2014, the "" Burial and Ex-gratia P...","Funeral Assistance: ₹3,000 payable in case of ...",The deceased construction worker should have b...,Step 1: The interested applicant should visit ...,Aadhaar Card of the applicant (nominee/Legal h...,State,Social welfare & Empowerment,"Building Worker, Construction Workers, Unregis..."
3,Consortia & Tender Marketing Scheme,ctms,Promotion of the product of Micro and Small En...,Enlistment of the Unit for participating in Go...,Micro & Small Enterprises registered with NSIC...,"Step 01: The application form, in the prescrib...",A passport size photograph of each of the Prop...,Central,Business & Entrepreneurship,"Goods And Services Marketing, Turnkey Projects..."
4,Garuda Scheme for Funeral Expense,gsfe,Andhra Pradesh Brahmin Welfare Corporation (AB...,"Financial Assistance of ₹10,000/- for funeral ...",The applicant should be a close relative of th...,Registration and apply Step 01: Applicants hav...,Passport-size Photograph of the Applicant Aadh...,State,Social welfare & Empowerment,"Social Welfare, Financial Assistance, Deceased..."


In [3]:
df.drop_duplicates(subset=["slug"], keep="first", inplace=True)
df.reset_index(drop=True, inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3397 entries, 0 to 3396
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   scheme_name     3397 non-null   object
 1   slug            3397 non-null   object
 2   details         3397 non-null   object
 3   benefits        3397 non-null   object
 4   eligibility     3397 non-null   object
 5   application     3395 non-null   object
 6   documents       3386 non-null   object
 7   level           3397 non-null   object
 8   schemeCategory  3397 non-null   object
 9   tags            3368 non-null   object
dtypes: object(10)
memory usage: 265.5+ KB


In [5]:
df['application'] = df['application'].fillna("Not Available")
df['documents'] = df['documents'].fillna("Not Specified")
df['tags'] = df['tags'].fillna("No Tags")

In [6]:
df["content"] = (
    df["details"] + "\n\n" +
    "Benefits: " + df["benefits"] + "\n\n" +
    "Eligibility: " + df["eligibility"] + "\n\n" +
    "Application: " + df["application"] + "\n\n" +
    "Documents Required: " + df["documents"]
)

df["metadata"] = df.apply(lambda row: {
    "scheme_name": row["scheme_name"],
    'slug': row['slug'],
    "level": row["level"],
    "category": row["schemeCategory"],
    "tags": row["tags"]
}, axis=1)

print(f"Loaded CSV with {len(df)} rows")
df.head(5)

Loaded CSV with 3397 rows


,scheme_name,slug,details,benefits,eligibility,application,documents,level,schemeCategory,tags,content,metadata
0,"""Immediate Relief Assistance"" under ""Welfare a...",ira-wrflsncs,"The scheme ""Immediate Relief Assistance"" is a ...","₹ 1,00,000, in two installments of ₹ 50,000 ea...",The applicant should be the family (legal heir...,Step 1: The interested applicant should visit ...,Photograph of the Family (Legal Heir) of the M...,State,"Agriculture,Rural & Environment, Social welfar...","Missing, Fisherman, Relief, Financial Assistan...","The scheme ""Immediate Relief Assistance"" is a ...","{'scheme_name': '""Immediate Relief Assistance""..."
1,AICTE SHORT TERM TRAINING PROGRAMME-SFURTI SCHEME,astpss,"Short Term Training Programme-SFURTI Program, ...","Financial Assistance : Limit of funding ₹ 4,00...",The institution should be AICTE approved.,Registration of New Institute: Step 01: Visit ...,Feedback Form Copy of Proceedings Completion R...,Central,Education & Learning,"Trainings, Financial Assistance, AICTE","Short Term Training Programme-SFURTI Program, ...",{'scheme_name': 'AICTE SHORT TERM TRAINING PRO...
2,Burial and Ex-gratia Payment Scheme in Case of...,baepsicodouldwact,"Launched in 2014, the "" Burial and Ex-gratia P...","Funeral Assistance: ₹3,000 payable in case of ...",The deceased construction worker should have b...,Step 1: The interested applicant should visit ...,Aadhaar Card of the applicant (nominee/Legal h...,State,Social welfare & Empowerment,"Building Worker, Construction Workers, Unregis...","Launched in 2014, the "" Burial and Ex-gratia P...",{'scheme_name': 'Burial and Ex-gratia Payment ...
3,Consortia & Tender Marketing Scheme,ctms,Promotion of the product of Micro and Small En...,Enlistment of the Unit for participating in Go...,Micro & Small Enterprises registered with NSIC...,"Step 01: The application form, in the prescrib...",A passport size photograph of each of the Prop...,Central,Business & Entrepreneurship,"Goods And Services Marketing, Turnkey Projects...",Promotion of the product of Micro and Small En...,{'scheme_name': 'Consortia & Tender Marketing ...
4,Garuda Scheme for Funeral Expense,gsfe,Andhra Pradesh Brahmin Welfare Corporation (AB...,"Financial Assistance of ₹10,000/- for funeral ...",The applicant should be a close relative of th...,Registration and apply Step 01: Applicants hav...,Passport-size Photograph of the Applicant Aadh...,State,Social welfare & Empowerment,"Social Welfare, Financial Assistance, Deceased...",Andhra Pradesh Brahmin Welfare Corporation (AB...,{'scheme_name': 'Garuda Scheme for Funeral Exp...


### Chunking 

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,   
    chunk_overlap=400  
)

documents = []
for idx, row in df.iterrows():
    chunks = text_splitter.split_text(row["content"])
    for i, chunk in enumerate(chunks):
        if chunk.strip():  
            documents.append({
                "id": f"{row['slug']}_chunk{i+1}",
                "text": chunk,
                "metadata": row["metadata"]
            })

print(f"Split into {len(documents)} text chunks")

Split into 19880 text chunks


In [8]:
documents

[{'id': 'ira-wrflsncs_chunk1',
  'text': 'The scheme "Immediate Relief Assistance" is a Sub-Component under the scheme "Welfare and Relief for Fishermen During Lean Seasons and Natural Calamities Scheme". The scheme is extended to all the regions of the Union territory of Puducherry. The scheme is introduced with the objective of extending financial assistance to the fishermen\'s families to compensate for the loss due to the missing breadwinner and to support them financially to run their family.',
  'metadata': {'scheme_name': '"Immediate Relief Assistance" under "Welfare and Relief for Fishermen During Lean Seasons and Natural Calamities Scheme"',
   'slug': 'ira-wrflsncs',
   'level': 'State',
   'category': 'Agriculture,Rural & Environment, Social welfare & Empowerment',
   'tags': 'Missing, Fisherman, Relief, Financial Assistance, Family'}},
 {'id': 'ira-wrflsncs_chunk2',
  'text': 'Benefits: ₹ 1,00,000, in two installments of ₹ 50,000 each, as immediate relief assistance for the

### Embeddings & Vector store

In [9]:
embeddings = SentenceTransformerEmbeddings(
    model_name="BAAI/bge-large-en-v1.5"  
)

In [10]:
docs_with_vectors = [
    Document(page_content=doc["text"], metadata=doc["metadata"])
    for doc in tqdm(documents, desc="Preparing documents...")
]

ids = [doc["id"] for doc in documents]
texts = [doc.page_content for doc in docs_with_vectors]
metadatas = [doc.metadata for doc in docs_with_vectors]

Preparing documents...: 100%|██████████| 19880/19880 [00:00<00:00, 307917.26it/s]


In [11]:
'''chunk wise embedding and store it in chroma db'''

batch_size = 256
precomputed_embeddings = []

for i in tqdm(range(0, len(texts), batch_size), desc="Embedding documents..."):
    batch_texts = texts[i : i + batch_size]
    batch_embeds = embeddings.embed_documents(batch_texts)
    precomputed_embeddings.extend(batch_embeds)

precomputed_embeddings = np.array(precomputed_embeddings).tolist()

Embedding documents...: 100%|██████████| 78/78 [43:15<00:00, 33.28s/it] 


In [12]:
'''save embeddings'''

np.save("precomputed_embeddings.npy", precomputed_embeddings)

'''.......................................................................................................'''

with open("precomputed_embeddings.pkl", "wb") as f:
    pickle.dump(precomputed_embeddings, f)

In [13]:
'''load embeddings'''


precomputed_embeddings = np.load("precomputed_embeddings.npy", allow_pickle=True)
precomputed_embeddings = precomputed_embeddings.tolist()

print(f"Loaded {len(precomputed_embeddings)} embeddings")

'''.......................................................................................................'''

# with open("precomputed_embeddings.pkl", "rb") as f:
#     precomputed_embeddings = pickle.load(f)

# print(f"Loaded {len(precomputed_embeddings)} embeddings")

Loaded 19880 embeddings


'.......................................................................................................'

In [14]:
'''uniqueness check'''

assert len(ids) == len(texts) == len(metadatas) == len(precomputed_embeddings), \
    f"Length mismatch! ids={len(ids)}, texts={len(texts)}, metadatas={len(metadatas)}, embeddings={len(precomputed_embeddings)}"

if len(ids) != len(set(ids)):
    raise ValueError("Duplicate IDs detected! Please fix before inserting into Chroma.")

In [15]:
os.makedirs("./chroma_store", exist_ok=True)
client = chromadb.PersistentClient(path="./chroma_store")
collection = client.get_or_create_collection(name="schemes_db")

max_batch_size = 5000  
for i in tqdm(range(0, len(texts), max_batch_size), desc="Adding to Chroma..."):
    collection.add(
        documents=texts[i : i + max_batch_size],
        metadatas=metadatas[i : i + max_batch_size],
        ids=ids[i : i + max_batch_size],
        embeddings=precomputed_embeddings[i : i + max_batch_size],
    )

print(f"Chroma vector store created with {len(documents)} documents")

Adding to Chroma...: 100%|██████████| 4/4 [00:09<00:00,  2.40s/it]

Chroma vector store created with 19880 documents


### Retriever

In [16]:
vectordb = Chroma(
    persist_directory="./chroma_store",
    embedding_function=embeddings,  
    collection_name="schemes_db"
)

retriever = vectordb.as_retriever(search_kwargs={"k": 3})

### Retrival testing with similarity

In [ ]:
query = "Eligibility criteria for Immediate Relief Assistance"
docs = retriever.get_relevant_documents(query)

for d in docs:
    print(d.page_content[:500], "...\n")  


The scheme "Immediate Relief Assistance" is a Sub-Component under the scheme "Welfare and Relief for Fishermen During Lean Seasons and Natural Calamities Scheme". The scheme is extended to all the regions of the Union territory of Puducherry. The scheme is introduced with the objective of extending financial assistance to the fishermen's families to compensate for the loss due to the missing breadwinner and to support them financially to run their family. ...

The "Immediate Assistance in Case of Accident to the Beneficiary" scheme was launched by the Manipur Building & Other Construction Workers' Welfare Board. The scheme aims to provide financial assistance to registered building workers in cases of accidents, disabilities, or death due to accidents. Workers hospitalized for five or more days due to an accident are given immediate assistance, and further aid is available for those who suffer disabilities or death resulting from such accidents. ...

"Interim Relief to the Beneficiarie

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
def jaccard_similarity(text1, text2):
    a = set(text1.lower().split())
    b = set(text2.lower().split())
    return len(a & b) / len(a | b)

original_text = """Benefits
₹ 1,00,000, in two installments of ₹ 50,000 each, as immediate relief assistance for the family (legal heir) of the missing fisherman.
Disbursal
Initially, 50% will be extended within 3 months from the date of receipt of the application from the family (legal heir).
The family (legal heir) should approach this department for the release of the balance 50% of the relief which will be deposited in the bank in a joint account in the name of kin (legal heir) and the competent authority concerned.
If no further information is received about the missing person, the balance amount will be released in favour of the next of kin (legal heir), after the prescribed period of 9 months from the date of release of 1st part of lump sum.
*In case of the return of the missing fishermen, the amount extended as compensation either ₹ 50,000 or ₹ 1,00,000 as the case may be, will be recovered by invoking an insurance bond. 
Eligibility
The applicant should be the family (legal heir) of the missing fisherman.
The missing fisherman should have been a resident of the Union territory of Puducherry.
The missing fisherman must have lost his/her life while fishing. 
The missing fisherman must have been in the age group of 18-60 years.
The missing fisherman must not have been a beneficiary of the old age pension scheme. 
The missing fisherman should have enrolled as a member of the Fishermen/Fisherwomen Co-operative Society."""
retrieved_text = """Benefits: An immediate relief of up to ₹10,00,000/-. *The committee shall decide about the modalities regarding disbursement of such relief, quantum and procedure to be followed for such disbursement.

Eligibility: The applicant should be a beneficiary of a scheme of the Goa Building and Other Construction Workers Welfare Board. The applicant should be affected by disaster/catastrophe/natural calamity. ...

The scheme "Immediate Relief Assistance" is a Sub-Component under the scheme "Welfare and Relief for Fishermen During Lean Seasons and Natural Calamities Scheme". The scheme is extended to all the regions of the Union territory of Puducherry. The scheme is introduced with the objective of extending financial assistance to the fishermen's families to compensate for the loss due to the missing breadwinner and to support them financially to run their family. ...

The "Immediate Assistance in Case of Accident to the Beneficiary" scheme was launched by the Manipur Building & Other Construction Workers' Welfare Board. The scheme aims to provide financial assistance to registered building workers in cases of accidents, disabilities, or death due to accidents. Workers hospitalized for five or more days due to an accident are given immediate assistance, and further aid is available for those who suffer disabilities or death resulting from such accidents. ..."""

similarity = jaccard_similarity(original_text, retrieved_text)
print(f"Jaccard similarity: {similarity:.2f}")  # 0.0–1.0 scale


orig_vec = embeddings.embed_documents([original_text])
retrieved_vec = embeddings.embed_documents([retrieved_text])

sim_score = cosine_similarity(orig_vec, retrieved_vec)[0][0]
print(f"Semantic similarity: {sim_score:.2f}")

Jaccard similarity: 0.12
Semantic similarity: 0.77


### Prompting

In [19]:
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a precise, friendly assistant specialized in Indian government schemes. Use ONLY the provided context; do NOT invent facts.

Decide automatically which **mode** to use based on the user input:

---

### MODE A — USER PROFILING → PERSONALIZED SCHEMES
- Trigger: user asks for “schemes for me,” “recommend schemes,” “which schemes can I apply for,” etc.
- Collect profile **step-by-step**, one question at a time:
    1. Name
    2. Age
    3. Gender
    4. Highest educational qualification
    5. Employment status (employed/self-employed/unemployed/student) and sector if employed
    6. State / UT
    7. Optional: special status (SC/ST/OBC/Woman/Person with Disability, etc.)
- After collecting all answers, retrieve **relevant schemes** from the context.
- Present each scheme in a **numbered mini-report** using the Output Format.
- If the user stops before completing the profile, ask the next question; do NOT fetch schemes until profile is complete or user explicitly requests.

---

### MODE B — DIRECT SCHEME QUERY
- Trigger: user input contains the **name of a scheme present in the context** or asks for details (eligibility, benefits, application, documents, duration).
- Search context for exact or close scheme names.
- Always produce the **Output Format**.
- Mark missing fields as `"Not available"`.
- NEVER ask the user to clarify the mode.

---

### CONTEXT USAGE
- Combine all context chunks to build a complete mini-report.
- If the same field appears in multiple chunks:
    - Identical values → use that value.
    - Conflicting values → list both, note "Conflict — multiple sources disagree."
- Include **all fields** if available: Benefits, Documents, Duration, etc.

---

### OUTPUT FORMAT
1. Scheme Name: <text or "Not available">
   Eligibility: <text or "Not available">
   Benefits: <text or "Not available">
   Application Process: <text or "Not available">
   Required Documents: <text or "Not available">
   Validity / Duration: <text or "Not available">
   Level (Central/State): <text or "Not available">
   Scheme Category: <text or "Not available">
   Tags: <text or "Not available">
   Sources:
     - <source identifier + fields provided>
   Completeness Score: <float 0.00-1.00>
   Confidence: <float 0.00-1.00>

Notes:
- NEVER hallucinate; only use context.
- Include source citations for each field.
- Use "Not available" when data is missing.
- ALWAYS return output in this format.

---

### FEW-SHOT EXAMPLES

Example A — Direct query:
User: "Tell me about Ayushman Bharat - PM-JAY eligibility and benefits."
Assistant:
1. Scheme Name: Ayushman Bharat - Pradhan Mantri Jan Arogya Yojana (AB PM-JAY)
   Eligibility: Rural/urban poor based on SECC 2011; bottom 40%; excludes high income/assets
   Benefits: Cashless coverage up to ₹5,00,000 per family/year for secondary & tertiary care
   Application Process: Offline via Arogya Mitra or CSC; e-card issued after verification
   Required Documents: PM letter / RSBY URN / Ration Card, Aadhaar or ID
   Validity / Duration: Not available
   Level (Central/State): Central
   Scheme Category: Health, Insurance
   Tags: HealthInsurance, Eligibility, SignInToApply
   Sources:
     - "AB PM-JAY" chunk (Eligibility, Benefits, Application Process, Documents)
   Completeness Score: 0.67
   Confidence: 0.90

Context:
{context}

User Input / Question:
{question}

Respond following the rules above and using the Output Format.
"""
)


### Rag Chain

In [20]:
llm = Ollama(
    model="qwen2.5:7b",
    temperature=0.7
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt},   
    return_source_documents=True
)

### Scoring------ LLM as a judge

In [21]:
llm_judge = Ollama(model="llama3.1:8b", temperature=0)

eval_prompt = PromptTemplate(
    input_variables=["question", "rag_answer", "sources"],
    template="""
You are a helpful and meticulous judge. Evaluate the RAG assistant's response to a user query.

User Question: {question}

RAG Response: {rag_answer}

Sources retrieved: {sources}

Tasks:
1. Give a **completeness score** between 0 and 1 based on how many of the following fields are addressed correctly: 
   Scheme Name, Eligibility, Benefits, Application Process, Required Documents, Validity / Duration, Level, Scheme Category, Tags.
2. Give a **confidence score** between 0 and 1 based on how correct and relevant the answer is with respect to the sources.
3. Provide a brief reasoning for your scores.

Output JSON format:
{{
    "completeness_score": float,
    "confidence_score": float,
    "reasoning": str
}}
"""
)

eval_chain = LLMChain(llm=llm_judge, prompt=eval_prompt)

def evaluate_answer_with_llm(result, question):
    """
    Use the LLM to judge the RAG response.
    """
    rag_answer = result.get("answer", "")
    sources = [doc.metadata.get("scheme_name", doc.metadata.get("source", "Unknown")) 
               for doc in result.get("source_documents", [])]

    evaluation = eval_chain.run(
        question=question,
        rag_answer=rag_answer,
        sources=", ".join(sources)
    )

    try:
        eval_scores = json.loads(evaluation)
        result["completeness_score"] = eval_scores.get("completeness_score", 0.0)
        result["confidence_score"] = eval_scores.get("confidence_score", 0.0)
        result["evaluation_reasoning"] = eval_scores.get("reasoning", "")
    except:
        result["completeness_score"] = 0.0
        result["confidence_score"] = 0.0
        result["evaluation_reasoning"] = evaluation  # fallback raw text

    return result


### Testing

In [22]:
def chat_with_rag():
    print("RAG Assistant is ready! Type your question about government schemes.")
    print("Type 'exit' to quit.\n")

    while True:
        print("Select mode: [1] Personalized schemes (profile), [2] Direct scheme query")
        mode = input("Enter 1 or 2: ").strip()
        if mode.lower() in ["exit", "quit", "bye"]:
            print("Assistant: Thank you! Have a great day.")
            break
        if mode not in ["1", "2"]:
            print("Invalid choice. Please enter 1 or 2.\n")
            continue

        if mode == "1":
            print("\nHello! Let's collect your info to suggest relevant schemes.\n")
            user_info = {}
            user_info["name"] = input("Your name: ").strip()
            user_info["age"] = input("Your age: ").strip()
            user_info["gender"] = input("Your gender: ").strip()
            user_info["education"] = input("Your highest educational qualification: ").strip()
            user_info["employment"] = input("Are you employed? If yes, which sector: ").strip()
            user_info["state"] = input("Which state/region do you live in: ").strip()
            query = f"Retrieve government schemes relevant to this user profile: {user_info}"
        else:
            query = input("\nEnter your question about a specific scheme: ").strip()
            if query.lower() in ["exit", "quit", "bye"]:
                print("Assistant: Thank you! Have a great day.")
                break

        try:
            result = rag_chain({"question": query})
        except Exception as e:
            print("Error retrieving answer:", e)
            continue

        try:
            result = evaluate_answer_with_llm(result, query)
        except Exception as e:
            print("Error evaluating answer:", e)
            continue

        print("\nAssistant:\n", result.get("answer", "No answer available."))

        if "source_documents" in result and result["source_documents"]:
            print("\nSources:")
            for i, doc in enumerate(result["source_documents"], 1):
                source_name = doc.metadata.get("scheme_name", doc.metadata.get("source", "Unknown"))
                print(f"  {i}. {source_name}")

        print("\nCompleteness Score:", result.get("completeness_score", 0.0))
        print("Confidence Score:", result.get("confidence_score", 0.0))
        print("-" * 80)

chat_with_rag()


RAG Assistant is ready! Type your question about government schemes.
Type 'exit' to quit.

Select mode: [1] Personalized schemes (profile), [2] Direct scheme query

Assistant:
 1. Scheme Name: Pradhan Mantri Kaushal Vikas Yojana - Short Term Training
   Eligibility: Not available
   Benefits: Stipend: 12th Pass - ₹6,000/- ITI / Diploma - ₹8,000/- Degree / Post Graduation - ₹10,000/-
   Application Process: Not available
   Required Documents: Not available
   Validity / Duration: Not available
   Level (Central/State): Central
   Scheme Category: Skill Development
   Tags: SkillDevelopment, Stipend
   Sources:
     - "Pradhan Mantri Dakshta Aur Kushalta Sampann Hitgrahi" chunk (Benefits)
   Completeness Score: 0.33
   Confidence: 0.85

Sources:
  1. Mukhyamantri Protsahan Yojana Jharkhand
  2. Mukhyamantri Yuva Karya Prashikshan Yojana
  3. Pradhan Mantri Dakshta Aur Kushalta Sampann Hitgrahi (PM-DAKSH)

Completeness Score: 0.0
Confidence Score: 0.0
------------------------------------